In [1]:
from data_loader import *
from __future__ import division
from __future__ import print_function
import pandas as pd
import bokeh
import numpy as np

import bokeh.plotting as bp
from bokeh.plotting import figure, output_file, show,gridplot
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.axes import DatetimeAxis

from datetime import datetime
from datetime import timedelta

import pandas as pd

bp.output_notebook()

Loading BokehJS ...

In [29]:
#data = DataLoader('E3_7E_44_77_F2_38.scn') #badge data from short experiment
data = DataLoader('FF_6B_D4_CB_22_FA.scn') #badge data from short experiment

session_start_time = datetime(2016,1,27,14,26,0)
session_end_time = datetime(2016,1,27,14,46,0)

#session_start_time = datetime(2016,1,27,15,10,0)
#session_end_time = datetime(2016,1,27,16,5,0)

#data = DataLoader('badge_v33_EC_21_82_A8_0B_59.scn') #badge data from a 4 days test

In [30]:
# take only samples
signals = map(lambda x: x['samples'], data.data)

# convert 
con_signals = [map(lambda x: {"datetime": x["time"], "signal": x['signal']}, i)  for i in signals]
#con_signals[0]

# Append all samples into an array
singals_all = []
for s in con_signals:
    singals_all.extend(s)
    
# sort
sdata = sorted(singals_all, key=lambda x: x['datetime'])

# Check for gaps in data (before loading into pandas. It's faster)

In [31]:
def check_raw_gaps(arr, max_delta_sec = 15):
    #max_diff = np.timedelta64(max_delta_sec, 's')
    max_diff = timedelta(seconds=1);
    prev = None
    for i in range (0,len(arr)): # iterate over all lines
        #print df.iloc[i].datetime
        curr = arr[i]['datetime']
        if (prev):
            diff = curr-prev
            if diff > max_diff:
                print ("{},{},{}".format(curr,prev,diff))
                #break
        prev = curr

In [32]:
check_raw_gaps(sdata,15)

2016-01-27 12:13:25,2016-01-26 23:18:01.750000,12:55:23.250000
2016-01-27 15:14:40,2016-01-27 15:14:10.750000,0:00:29.250000
2016-01-27 15:39:48,2016-01-27 15:39:18.750000,0:00:29.250000
2016-01-27 17:01:27,2016-01-27 17:00:01.750000,0:01:25.250000
2016-01-27 17:03:31,2016-01-27 17:01:55.750000,0:01:35.250000


# Loading data infor Pandas

In [33]:
df = pd.DataFrame()

In [34]:
sdata[10:20]

[{'datetime': datetime.datetime(2016, 1, 26, 23, 17, 35, 500000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 35, 750000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 36), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 36, 250000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 36, 500000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 36, 750000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 37), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 37, 250000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 37, 500000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 17, 37, 750000), 'signal': 1}]

In [35]:
# Append all samples into a DataFrame
#for s in sdata:
df = df.append(pd.DataFrame(sdata))

In [36]:
df = df.set_index(pd.DatetimeIndex(df['datetime']))
#df.index

In [37]:
df_fil = df.ix[session_start_time:session_end_time]

In [38]:
df_fil[0:1]

,datetime,signal
2016-01-27 14:26:00,2016-01-27 14:26:00,1


In [39]:
tools = "xwheel_zoom,xpan,reset,resize,save"
p = figure(title="Signal", x_axis_label='x', y_axis_label='y'
           ,x_axis_type="datetime",width=1000, height=400,y_range=[0, 20], tools=tools)
# add a line renderer with legend and line thickness
p.line(df_fil.index.to_pydatetime(), df.signal, legend="Raw", line_width=2)
p.below[0].formatter.formats = dict(years=['%Y'],months=['%b %Y'],days=['%d %b %Y'],hours=["%I:%M %P"],minutes=["%I:%M %P"]) #minutes is the one i'm interested in

# show the results
show(p)

# #Look for gaps

In [13]:
def check_individual_data_gaps(df, max_delta_sec = 15):
	max_diff = np.timedelta64(max_delta_sec, 's')
	prev = None
	for i in range (0,len(df)): # iterate over all lines
		#print df.iloc[i].datetime
		curr = df.iloc[i].time
		if (prev):
			diff = curr-prev
			if diff > max_diff:
				print ("{},{},{}".format(curr,prev,diff))
				#break
		prev = curr

In [14]:
'''
check_individual_data_gaps(df_fil,15)
'''

'\ncheck_individual_data_gaps(df_fil,15)\n'